## IMPORTING LIBRARIES

In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import gradio as gr
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

## LOAD AND PRE-PROCESS DATA 

In [2]:
IMG_SIZE = 224
TRAIN_DIR = 'Training'
TEST_DIR = 'Testing'
CATEGORIES = ['glioma', 'meningioma', 'notumor', 'pituitary']

# --- Function to Load and Preprocess Data ---
def load_data(directory):
    data = []
    labels = []
    for category in CATEGORIES:
        path = os.path.join(directory, category)
        for img_name in os.listdir(path):
            img_path = os.path.join(path, img_name)
            try:
                img = cv2.imread(img_path)
                img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                data.append(img)
                labels.append(category)
            except Exception as e:
                print(f"Error loading image {img_path}: {e}")
    return np.array(data), np.array(labels)

print("[INFO] Loading training data...")
X_train, y_train = load_data(TRAIN_DIR)

print("[INFO] Loading testing data...")
X_test, y_test = load_data(TEST_DIR)

X_train = X_train / 255.0
X_test = X_test / 255.0

le = LabelEncoder()
y_train_enc = to_categorical(le.fit_transform(y_train))
y_test_enc = to_categorical(le.transform(y_test))

[INFO] Loading training data...
[INFO] Loading testing data...


## BUILD CNN MODEL

In [3]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')  
])

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()



D:\Anaconda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │      11,075,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4)                   │             516 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,169,476 (42.61 MB)

 Trainable params: 11,169,476 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

## TRAIN MODEL 

In [4]:
# --- Train Model ---
print("[INFO] Training model...")
history = model.fit(X_train, y_train_enc, epochs=20, validation_data=(X_test, y_test_enc), batch_size=32)

# Save the model
model.save("brain_tumor_model.h5")
print("[INFO] Model saved as brain_tumor_model.h5")

# --- Evaluate Model ---
test_loss, test_acc = model.evaluate(X_test, y_test_enc)
print(f"✅ Test Accuracy: {test_acc * 100:.2f}%")

# --- Gradio Interface Functions ---
model = load_model("brain_tumor_model.h5")



def preprocess_image(img):
    # Handle grayscale or alpha channel
    if len(img.shape) == 2:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    elif img.shape[2] == 1:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    elif img.shape[2] == 4:
        img = cv2.cvtColor(img, cv2.COLOR_RGBA2RGB)

    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    return img


def predict_mri(image):
    try:
        preprocessed = preprocess_image(image)
        prediction = model.predict(preprocessed)[0]
        predicted_class = CATEGORIES[np.argmax(prediction)]
        confidence = np.max(prediction) * 100
        analytics = {CATEGORIES[i]: f"{prediction[i]*100:.2f}%" for i in range(len(CATEGORIES))}
        return predicted_class, confidence, analytics
    except Exception as e:
        return "Error", 0, {"Error": str(e)}

[INFO] Training model...
Epoch 1/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 472s 3s/step - accuracy: 0.5107 - loss: 1.0566 - val_accuracy: 0.7651 - val_loss: 0.6403
Epoch 2/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 512s 3s/step - accuracy: 0.7990 - loss: 0.5368 - val_accuracy: 0.8169 - val_loss: 0.4580
Epoch 3/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 509s 3s/step - accuracy: 0.8454 - loss: 0.4249 - val_accuracy: 0.8467 - val_loss: 0.3948
Epoch 4/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 501s 3s/step - accuracy: 0.8720 - loss: 0.3582 - val_accuracy: 0.8558 - val_loss: 0.3505
Epoch 5/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 522s 3s/step - accuracy: 0.8931 - loss: 0.3180 - val_accuracy: 0.8642 - val_loss: 0.3524
Epoch 6/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 484s 3s/step - accuracy: 0.9089 - loss: 0.2480 - val_accuracy: 0.8863 - val_loss: 0.2746
Epoch 7/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 496s 3s/step - accuracy: 0.9215 - loss: 0.2127 - val_accuracy: 0.8780 - val_loss: 0.3082
Epoch 8/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 481s 3s/step - accuracy: 0.9264 

[INFO] Model saved as brain_tumor_model.h5
41/41 ━━━━━━━━━━━━━━━━━━━━ 28s 692ms/step - accuracy: 0.9320 - loss: 0.2136
✅ Test Accuracy: 94.58%


## GRADIO INTERFACE

In [7]:
def gradio_interface(image):
    prediction, confidence, analytics = predict_mri(image)
    if prediction == "Error":
        return f"⚠️ {analytics['Error']}"
    
    output_text = f"Prediction: {prediction}\nConfidence: {confidence:.2f}%\n\nAnalytics:\n"
    for label, percent in analytics.items():
        output_text += f"{label}: {percent}\n"
    return output_text

# --- Launch Gradio ---
interface = gr.Interface(
    fn=gradio_interface,
    inputs=gr.Image(type="numpy", label="Upload MRI Image"),
    outputs="text",
    title="🧠 MRI Brain Tumor Detection",
    description="Upload an MRI scan to detect brain tumors (Glioma, Meningioma, No Tumor, Pituitary)."
)

interface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://2709be38ea736d59d2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
